# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [1]:
# import libraries

# libraries for data manipulation & loading
import pandas as pd
import numpy as np
from sqlalchemy import create_engine
import pickle

# libraries for NLP
import re
import nltk
from nltk.corpus import stopwords
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.tokenize import word_tokenize

nltk.download(['punkt', 'stopwords', 'wordnet', 'averaged_perceptron_tagger'])

# libraries for ML
from sklearn.multioutput import MultiOutputClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer, TfidfVectorizer
from sklearn.metrics import classification_report
from sklearn.naive_bayes import MultinomialNB


[nltk_data] Downloading package punkt to
[nltk_data]     /Users/sebastianminaya/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/sebastianminaya/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/sebastianminaya/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /Users/sebastianminaya/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


In [2]:
# load data from database
engine = create_engine('sqlite:///etl_done.db')
df = pd.read_sql_table('fact_messages',engine)
X = df['message'].values
Y = df.iloc[:,4:].values
categories = df.iloc[:,4:].columns
print(X.shape, Y.shape)

(26215,) (26215, 36)


### 2. Write a tokenization function to process your text data

In [3]:
def tokenize(text):
    # instantiate the stopwords list & lemmatizer
    stop_words = stopwords.words("english")
    lemmatizer = WordNetLemmatizer()
    
    # normalize case and remove punctuation
    text = re.sub(r"[^a-zA-Z0-9]", " ", text.lower())
    
    # tokenize text
    tokens = word_tokenize(text)
    
    # lemmatize andremove stop words
    tokens = [lemmatizer.lemmatize(word) for word in tokens if word not in stop_words]

    return tokens

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [4]:
pipeline = Pipeline([
        ('vect', CountVectorizer(tokenizer=tokenize)),
        ('tfidf', TfidfTransformer()),
        ('clf', MultiOutputClassifier(RandomForestClassifier()))
])

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [5]:
X_train, X_test, y_train, y_test = train_test_split(X, Y)

pipeline.fit(X_train, y_train)

Pipeline(steps=[('vect',
                 CountVectorizer(tokenizer=<function tokenize at 0x16d062830>)),
                ('tfidf', TfidfTransformer()),
                ('clf',
                 MultiOutputClassifier(estimator=RandomForestClassifier()))])

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [6]:
y_pred = pipeline.predict(X_test)

for i, element in enumerate(np.transpose(y_pred)):
    print(categories[i])
    print(classification_report(np.transpose(y_pred)[i], np.transpose(y_test)[i]))

related
              precision    recall  f1-score   support

           0       0.40      0.69      0.51       859
           1       0.95      0.84      0.89      5695

    accuracy                           0.82      6554
   macro avg       0.67      0.77      0.70      6554
weighted avg       0.88      0.82      0.84      6554

request
              precision    recall  f1-score   support

           0       0.98      0.91      0.94      5870
           1       0.51      0.84      0.64       684

    accuracy                           0.90      6554
   macro avg       0.75      0.87      0.79      6554
weighted avg       0.93      0.90      0.91      6554

offer
              precision    recall  f1-score   support

           0       1.00      1.00      1.00      6554
           1       0.00      0.00      0.00         0

    accuracy                           1.00      6554
   macro avg       0.50      0.50      0.50      6554
weighted avg       1.00      1.00      1.00      655

/Users/sebastianminaya/miniconda3/envs/disaster_response_ml/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1308: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/sebastianminaya/miniconda3/envs/disaster_response_ml/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1308: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/sebastianminaya/miniconda3/envs/disaster_response_ml/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1308: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavi

### 6. Improve your model
Use grid search to find better parameters. 

In [13]:
parameters = {'clf__estimator__criterion': ["gini", "entropy"],     
        'clf__estimator__n_jobs':[-1]
        }

cv = GridSearchCV(pipeline, parameters)

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [14]:
# fit the pipeline
cv.fit(X_train, y_train)

# make predictions
y_pred = cv.predict(X_test)

# print classification report
for i, element in enumerate(np.transpose(y_pred)):
    print(categories[i])
    print(classification_report(np.transpose(y_pred)[i], np.transpose(y_test)[i]))

related
              precision    recall  f1-score   support

           0       0.39      0.69      0.49       837
           1       0.95      0.84      0.89      5717

    accuracy                           0.82      6554
   macro avg       0.67      0.76      0.69      6554
weighted avg       0.88      0.82      0.84      6554

request
              precision    recall  f1-score   support

           0       0.98      0.91      0.94      5868
           1       0.50      0.82      0.62       686

    accuracy                           0.90      6554
   macro avg       0.74      0.86      0.78      6554
weighted avg       0.93      0.90      0.91      6554

offer
              precision    recall  f1-score   support

           0       1.00      1.00      1.00      6554
           1       0.00      0.00      0.00         0

    accuracy                           1.00      6554
   macro avg       0.50      0.50      0.50      6554
weighted avg       1.00      1.00      1.00      655

/Users/sebastianminaya/miniconda3/envs/disaster_response_ml/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1308: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/sebastianminaya/miniconda3/envs/disaster_response_ml/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1308: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/sebastianminaya/miniconda3/envs/disaster_response_ml/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1308: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavi

### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

In [16]:
pipeline_ = Pipeline([
        #('vect', CountVectorizer(tokenizer=tokenize)),
        #('tfidf', TfidfTransformer()),
        ('tfidf', TfidfVectorizer(tokenizer=tokenize)),
        ('clf', MultiOutputClassifier(RandomForestClassifier()))
        #('clf', MultiOutputClassifier(MultinomialNB()))
])

cv_ = GridSearchCV(pipeline_, parameters)

cv_.fit(X_train, y_train)

y_pred = cv_.predict(X_test)

for i, element in enumerate(np.transpose(y_pred)):
    print(categories[i])
    print(classification_report(np.transpose(y_pred)[i], np.transpose(y_test)[i]))

related
              precision    recall  f1-score   support

           0       0.39      0.69      0.50       847
           1       0.95      0.84      0.89      5707

    accuracy                           0.82      6554
   macro avg       0.67      0.76      0.69      6554
weighted avg       0.88      0.82      0.84      6554

request
              precision    recall  f1-score   support

           0       0.98      0.91      0.94      5873
           1       0.51      0.84      0.63       681

    accuracy                           0.90      6554
   macro avg       0.75      0.87      0.79      6554
weighted avg       0.93      0.90      0.91      6554

offer
              precision    recall  f1-score   support

           0       1.00      1.00      1.00      6554
           1       0.00      0.00      0.00         0

    accuracy                           1.00      6554
   macro avg       0.50      0.50      0.50      6554
weighted avg       1.00      1.00      1.00      655

/Users/sebastianminaya/miniconda3/envs/disaster_response_ml/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1308: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/sebastianminaya/miniconda3/envs/disaster_response_ml/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1308: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/sebastianminaya/miniconda3/envs/disaster_response_ml/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1308: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavi

### 9. Export your model as a pickle file

In [17]:
filename = 'classifier.pkl'
pickle.dump(cv_, open(filename, 'wb'))

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.